In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from IPython.display import Image 
import io
from sklearn import tree
# import pydotplus
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
# from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
# import graphviz
# import pydotplus
import io
from scipy import misc
from sklearn.metrics import mean_squared_error
%matplotlib inline
import joblib

In [5]:
data = pd.read_csv('NewSl.csv')
print(data.head(3), '\n...\n...\n...\n',data.tail(2))

   Date  Month  Year   RF  T max   T min  Avg_tem
0     1     10  2013  0.0    31.1   25.8    28.45
1     2     10  2013  0.0    31.3   27.5    29.40
2     3     10  2013  1.6    31.3   27.3    29.30 
...
...
...
       Date  Month  Year    RF  T max   T min  Avg_tem
3650    29      9  2023  60.3    29.8   23.8    26.80
3651    30      9  2023  29.6    30.4   23.5    26.95


In [6]:
data.dtypes

Date         int64
Month        int64
Year         int64
RF         float64
T max      float64
T min      float64
Avg_tem    float64
dtype: object

In [7]:
pd.to_numeric(data['Avg_tem'])

0       28.45
1       29.40
2       29.30
3       28.20
4       27.60
        ...  
3647    28.85
3648    27.50
3649    26.65
3650    26.80
3651    26.95
Name: Avg_tem, Length: 3652, dtype: float64

In [8]:
rain = data['RF']
temp = data['Avg_tem']

In [9]:
print(rain)

0        0.0
1        0.0
2        1.6
3        5.8
4        0.3
        ... 
3647    18.4
3648    42.4
3649    63.4
3650    60.3
3651    29.6
Name: RF, Length: 3652, dtype: float64


In [10]:
print(temp)

0       28.45
1       29.40
2       29.30
3       28.20
4       27.60
        ...  
3647    28.85
3648    27.50
3649    26.65
3650    26.80
3651    26.95
Name: Avg_tem, Length: 3652, dtype: float64


In [11]:
data['Year'] = data.Year.astype(str)
data['Month'] = data.Month.astype(str)
data['Date'] = data.Date.astype(str)

data['date'] = data['Year'].str.cat(data['Month'], sep = '/')
data['DATE'] = data['date'].str.cat(data['Date'], sep = '/')
data.drop(columns=['Year','Month','Date','date'],axis=1,inplace=True)
data.set_index(['DATE'], inplace = True)
data.head()

RF  T max   T min  Avg_tem
DATE                                  
2013/10/1  0.0    31.1   25.8    28.45
2013/10/2  0.0    31.3   27.5    29.40
2013/10/3  1.6    31.3   27.3    29.30
2013/10/4  5.8    30.7   25.7    28.20
2013/10/5  0.3    30.8   24.4    27.60

In [12]:
data.dtypes

RF         float64
T max      float64
T min      float64
Avg_tem    float64
dtype: object

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data.reset_index(inplace=True)
data["DATE"] = label_encoder.fit_transform(data["DATE"])
print(data)

      DATE    RF  T max   T min  Avg_tem
0        0   0.0    31.1   25.8    28.45
1       11   0.0    31.3   27.5    29.40
2       22   1.6    31.3   27.3    29.30
3       25   5.8    30.7   25.7    28.20
4       26   0.3    30.8   24.4    27.60
...    ...   ...     ...    ...      ...
3647  3640  18.4    31.3   26.4    28.85
3648  3641  42.4    29.8   25.2    27.50
3649  3642  63.4    28.7   24.6    26.65
3650  3643  60.3    29.8   23.8    26.80
3651  3645  29.6    30.4   23.5    26.95

[3652 rows x 5 columns]


In [14]:
print(data.columns)

Index(['DATE', 'RF', 'T max ', 'T min', 'Avg_tem'], dtype='object')


In [15]:
inputs = ["T max " , "T min" , "Avg_tem" , "DATE"]

In [16]:
output = ["RF"]

In [17]:
X = data[inputs]
Y = data[output]

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, random_state=42)

In [19]:
model = RandomForestRegressor(n_estimators=200,random_state=42)
model.fit(X_train,Y_train)
Y_prediction = model.predict(X_test)
def rmse(a, b):
    return mean_squared_error(a, b, squared=False)

training_value = rmse(model.predict(X_train),Y_train)
testing_value = rmse(model.predict(X_test),Y_test)

print(f'Training RMSE: {training_value}')
print(f'Testing RMSE: {testing_value}')

c:\Users\Hammaad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training RMSE: 6.150963825856136
Testing RMSE: 16.185429928221073


c:\Users\Hammaad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Hammaad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [20]:
import pickle
pickle.dump(model, open('weatherModel.pkl','wb'))

In [202]:
# Save the trained model to a file using joblib
model_RainPredict = 'weather_prediction_model.joblib'
joblib.dump(model, model_RainPredict)
print(f'Model saved to {model_RainPredict}')

Model saved to weather_prediction_model.joblib


In [21]:
day = int(input('Enter Day'))
month = int(input('Enter Month'))
year = int(input('Enter Year'))
T_Min = float(input('Enter Minimum Temp'))
T_Max = float(input('Enter Maximum Temp'))

In [23]:
T_Avg = (T_Min+T_Max)/2
print(T_Avg)

from datetime import datetime

# Combine inputs into a datetime object
user_date = datetime(year, month, day)
print(user_date)

# Convert datetime to float
float_representation = user_date.timestamp()

print(f'User entered date in float representation: {float_representation}')


# year = year.astype(str)
# month = month.astype(str)
# day = day.astype(str)

# Date = year.str.cat(month, sep = '/')
# DATE = day.str.cat(Date, sep = '/')
# # drop('year','month','Date','day'],axis=1,inplace=True)
# # data.set_index(['DATE'], inplace = True)
# print(DATE)

33.0
2024-02-13 00:00:00
User entered date in float representation: 1707762600.0


In [24]:
# Create a DataFrame for user input
user_data = pd.DataFrame({
#     'RF': [0],  # You may assume 0 rainfall for simplicity
    'T max ': [T_Max],
    'T min': [T_Min],
    'Avg_tem': [T_Avg],
    'DATE' : [float_representation],
})

In [25]:
loaded_model = pickle.load(open('weatherModel.pkl','rb'))
# Make a prediction
user_prediction = loaded_model.predict(user_data)
print(f'Predicted Average RainFall: {user_prediction[0]:.2f}')

Predicted Average RainFall: 60.71


In [265]:
# Load the saved model
loaded_model = joblib.load(model_RainPredict)

# Make a prediction using user input
user_prediction = loaded_model.predict(user_data)
print(f'Predicted Average RainFall: {user_prediction[0]:.2f}')

Predicted Average RainFall: 49.91
